# In Class Lab 1

### Due by midnight, thursday in your github repository 'Labs/Lab1' folder


In [4]:
# Import Modules 
import numpy as np # import numpy
import astropy.units as u # import astropy units
from astropy import constants as const # import astropy constants

### Astropy Units:  https://docs.astropy.org/en/stable/units/index.html


## Part A:  The Local Standard of Rest
Proper motion of Sgr A* from Reid & Brunthaler 2004
$\mu = 6.379$ mas/yr 

Peculiar motion of the sun, $v_\odot$ = 12.24 km/s  (Schonrich 2010)


$v_{tan} = 4.74 \frac{\mu}{\rm mas/yr} \frac{R_o}{\rm kpc} = V_{LSR} + v_\odot$


### a)

Create a function called VLSR to compute the local standard of res (V$_{LSR}$).

The function should take as input: the solar radius (R$_o$), the proper motion (mu)
and the peculiar motion of the sun in the $v_\odot$ direction.

Compute V$_{LSR}$ using three different values R$_o$: 
1. Water Maser Distance for the Sun :  R$_o$ = 8.34 kpc   (Reid 2014 ApJ 783) 
2. GRAVITY Collaboration Distance for the Sun:  R$_o$ = 8.178 kpc   (Abuter+2019 A&A 625)
3. Value for Distance to Sun listed in Sparke & Gallagher : R$_o$ = 7.9 kpc 


In [7]:
def VLSR(Ro, mu = 6.379, v_sun = 12.24*u.km/u.s):
    """ This funcition will compute the velocity at the local standard of rest
        VLSR = 4.74*mu*Ro - v_sun
    Inputs: Ro (astropy units kpc) = Distance from the Sun to the Galactic Center
            mu = Proper motion of Sag A* (mas/yr)
                Default is from Reid & Brunthaler 2004
            v_sun (kms/sec) = Peculiar motion of the Sun in the v direction (Schonrich+2010)
    Outputs: VLSR (astropy units km/s) The local standard of rest
    """
    VLSR = 4.74*(mu)*(Ro/u.kpc)*u.km/u.s-(v_sun)
    return VLSR


In [8]:
#Different values for the distance of the Sun to the galactic center
RoReid = 8.34*u.kpc #Reid + 2014
RoAbuter = 8.178*u.kpc #Abuter + 2019
RoSparke = 7.9*u.kpc #Sparke & Gallagher

In [25]:
#Compute VLSR using Reid 2014
VLSR_Reid = VLSR(RoReid)
print(f'Reid. {VLSR_Reid}')

#Compute VLSR using Abuter 2019
VLSR_Abuter = VLSR(RoAbuter)
print(f'Abuter. {VLSR_Abuter:.2f}')

#Compute VLSR using Sparke & Gallagher
VLSR_Sparke = VLSR(RoSparke)
print(f'Sparke. {VLSR_Sparke}')

Reid. 239.9320764 km / s
Abuter. 235.03 km / s
Sparke. 226.628034 km / s


### b)

compute the orbital period of the sun in Gyr using R$_o$ from the GRAVITY Collaboration (assume circular orbit)

Note that 1 km/s $\sim$ 1kpc/Gyr

In [29]:
#Orbital period = 2piR/V

def TorbSun(Ro, Vc):
    '''
    This is a function the computes the orbital period of the Sun
        T = 2*pi*R/V
    Inputs:
        Ro: astropy quantity, distance to the galactic center from the Sun (kpc)
        Vc: astropy quantity, velocity of the Sun around the galactic center in 
            the v-direction(km/s)
    Outputs:
        T: astropy quantity, Orbital period (Gyr)
    '''
    V_kpc_Gyr = Vc.to(u.kpc/u.Gyr) #Converting Vc(km/s) to Vc(kpc/Gyr)

    T = 2*np.pi*Ro/V_kpc_Gyr #Orbital period
    return T

In [31]:
VsunPec = 12.24*u.km/u.s #peculiar motion

In [41]:
Vsun = VLSR_Abuter + VsunPec #Total motion of the Sun in the v-direction

In [45]:
#Orbital period of the Sun
T_Abuter = np.around(TorbSun(RoAbuter, Vsun), 3)
print(T_Abuter)

0.203 Gyr


### c)

Compute the number of rotations about the GC over the age of the universe (13.8 Gyr)

In [55]:
#Number of rotations about the GC over the age of the Universe
AgeUniverse = 13.8*u.Gyr #Age of the Universe
print(f'{AgeUniverse/T_Abuter:.2f} rotations')

67.98 rotations


## Part B  Dark Matter Density Profiles

### a)
Try out Fitting Rotation Curves 
[here](http://wittman.physics.ucdavis.edu/Animations/RotationCurve/GalacticRotation.html)


### b)


In the Isothermal Sphere model, what is the mass enclosed within the solar radius (R$_o$) in units of M$_\odot$? 

Recall that for the Isothermal sphere :
$\rho(r) = \frac{V_{LSR}^2}{4\pi G r^2}$

Where $G$ = 4.4985e-6 kpc$^3$/Gyr$^2$/M$_\odot$, r is in kpc and $V_{LSR}$ is in km/s

What about at 260 kpc (in units of  M$_\odot$) ? 

In [61]:
Grav = const.G.to(u.kpc**3/u.Gyr**2/u.Msun)
print(Grav)

4.498502151469554e-06 kpc3 / (solMass Gyr2)


In [63]:
# Density profile rho = VLSR^2/(4*pi*G*R^2)
# Mass (r) = Integrate rho dV
#           Integrate rho 4*pi*r^2*dr
#           Integrate VLSR^2/ (4*pi*G*R^2) * 4*pi*r^2*dr
#           Integrate VLSR^2/G dr
#           VLSR^2/G*r
def massIso(r, VLSR):
    '''
    Function computes the dark matter mass enclosed within a given distance, r, 
        assuming an Isothermal Sphere configuration, M(r) = VLSR^2/G*r
    Inputs:
        r (astropy quantity): distance from the GC (kpc)
        VLSR (astropy quantity): the velocity at the local Standard of Rest (km/s)
    Outputs:
        M (astropy quantity): mass enclosed within r (Msun)
    '''
    VLSRkpcGyr = VLSR.to(u.kpc/u.Gyr) #Translating to kpc/Gyr
    M = VLSRkpcGyr**2/Grav*r #Isothermal Sphere mass profile

    return M

In [69]:
#Compute mass enclosed within Ro (Gravity Collab)
mIsoSolar = massIso(RoAbuter, VLSR_Abuter)
print(f'{mIsoSolar:.2e}')

1.05e+11 solMass


In [73]:
#Compute the mass enclosed at 260 kpc
mIso260 = massIso(260*u.kpc, VLSR_Abuter)
print(f'{mIso260:.2e}')

3.34e+12 solMass


In [77]:
#Compute the mass enclosed at 260 kpc and VLSR = 220 km/s
mIso260 = massIso(260*u.kpc, 220*u.km/u.s)
print(f'{mIso260:.2e}')

2.93e+12 solMass


## c) 

The Leo I satellite is one of the fastest moving satellite galaxies we know. 


It is moving with 3D velocity of magnitude: Vtot = 196 km/s at a distance of 260 kpc (Sohn 2013 ApJ 768)

If we assume that Leo I is moving at the escape speed:

$v_{esc}^2 = 2|\Phi| = 2 \int G \frac{\rho(r)}{r}dV $ 

and assuming the Milky Way is well modeled by a Hernquist Sphere with a scale radius of $a$= 30 kpc, what is the minimum mass of the Milky Way (in units of M$_\odot$) ?  

How does this compare to estimates of the mass assuming the Isothermal Sphere model at 260 kpc (from your answer above)

In [79]:
# Potential for a Hernquist Sphere
#. Phi = -G*M/(r+a)

#Escape speed becomes:
#. v_esc^2 = 2*G*M/(r+a)

#rearrange for M
#. M = v_esc^2/2/G*(r+a)

def massHernVesc(v_esc, r, a=30*u.kpc):
    '''
    This function determines the total Dark Matter mass needed given an escape speed, assuming a Hernquist profile
    Inputs:
        v_esc (astropy quantity): escape speed (km/s)
        
        r (astropy quantity): distance from the GC (kpc)
        
        a (astropy quantity): the Hernquist scale length (kpc)

    Outputs:
        M (astropy quantity): mass within r (Msun)
    '''
    v_esckpcGyr = v_esc.to(u.kpc/u.Gyr) #Converting v_esc into kpc/Gyr

    M = v_esckpcGyr**2/2/Grav*(r+a) #Computing the Mass

    return M
    

In [81]:
V_leo = 196*u.km/u.s #Speed of Leo I Sohn et al.
r = 260*u.kpc

In [85]:
MLeoI = massHernVesc(V_leo, r)
print(f'{MLeoI:.2e}')

1.30e+12 solMass
